In [28]:
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.classify import ClassifierI
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from statistics import mode
from nltk.classify.scikitlearn import SklearnClassifier
import re
from TurkishStemmer import TurkishStemmer
from zemberek import TurkishMorphology, TurkishSentenceNormalizer, TurkishTokenizer

In [29]:
# Load and preprocess data
df = pd.read_excel('formasyon_veri_yanitlari.xlsx')
df = df.drop(df.columns[[0, 1]], axis=1)

# Extract text-label pairs
text_label_list = []
for column in df.columns:
    label = column.split('-')[0]
    for sentence in df[column].dropna():
        text_label_list.append((sentence, label))

text_label_df = pd.DataFrame(text_label_list, columns=['Text', 'Label'])
# shuffle your dataframe in-place and reset the index
text_label_df = text_label_df.sample(frac=1).reset_index(drop=True)

In [30]:
# Initialize Zemberek tools
tokenizer = TurkishTokenizer.DEFAULT
stop_words = set(stopwords.words("turkish"))
stemmer = TurkishStemmer()
morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)

# Text Preprocessing
def preprocess_text(text):
    # ^ Matches the start of the string
    # If the first character of the set is '^', all the characters that are not in the set will be matched
    # \ signals a special sequence
    # [] Used to indicate a set of characters
    # Remove non-alphanumeric
    # remove any new leading and trailing whitespace
    text = re.sub(r"[^\sa-zA-Z0-9ğüşöçıĞÜŞİÖÇ]", "", text)
    text = text.strip()
    
    # normalize
    # attempt to reduce its randomness, bringing it closer to a predefined “standard”
    text = normalizer.normalize(text.lower())
    print(text)
    
    # tokenize
    # it’s the task of cutting a text into pieces called tokens.
    tokens = tokenizer.tokenize(text)
    words = [
        stemmer.stem(token.content)
        for token in tokens
        if token.content not in stop_words
    ]
    return " ".join(words)

# Apply preprocessing
text_label_df['Processed_Text'] = text_label_df['Text'].apply(preprocess_text)

2025-01-02 19:17:50,623 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 10.765490531921387

ters v uçuşa başlayın
şunun için sonuç bul voleybol formasyonu
v yap bakayım
çizgi oluştur
ters v formasyonunu oluştur
voleybol oynamanız lazım acil
voleybol oyna
keşke drönelar vakıfbank fatma gibi voleybol oynasa
hadi voleybol oynayalım çocuklar
tüm gücünü ve zekanı kullanarak bir v harfi çiz
voleybol maçı yap
donlar bize nasıl ok başı uçtuğunuzu gösterin
kalem ucu oluştur
drone larin voleybol formasyonunda uçmasını sağla
v oluştur
konumları ok başına benzeyecek şekilde drone koy
v ol
voleybola hazır hale geçin
eda erdem in oynadığı spor formasyonunu çalıştır
çizgi şeklinde hat oluştur
voleybol oynayan drönelar da vardır görebilir miyim
ok başı formasyonu gerçekleşsin
düz bir çizgi çizer misin
v istiyorum
smaç vur
tüm ihalar ile voleybol sürü gösterisi yap
çizgi formasyonunu oluştur
voleybol oyunu şeklinde hareket et
v yap
don top olsun ve voleybo

In [31]:
all_words = []
for text in text_label_df['Processed_Text']:
    all_words.extend(text.split())

all_words_dist = nltk.FreqDist(all_words)
most_common_words = '\n'.join([f"{word}: {freq}" for word, freq in all_words_dist.most_common(15)])
print(f"Most common 15 words:\n{most_common_words}\n")

Most common 15 words:
formasyon: 133
v: 128
voleybol: 102
üçgen: 91
ok: 78
şekl: 78
oluş: 77
çizg: 73
baş: 62
bir: 53
yap: 49
drönelar: 49
çiz: 46
ters: 29
ol: 28



In [32]:
# find features
word_features = list(all_words_dist.keys())[:400]

def find_features(document):
    words = set(document.split())
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

feature_sets = [(find_features(text), label) for (text, label) in text_label_list]

In [33]:
# datasets
train_size = int(len(feature_sets) * 0.8)
training_set = feature_sets[:train_size]
testing_set = feature_sets[train_size:]

# Classifiers
classifiers = {
    "Naive Bayes": nltk.NaiveBayesClassifier.train(training_set),
    "MultinomialNB": SklearnClassifier(MultinomialNB()).train(training_set),
    "BernoulliNB": SklearnClassifier(BernoulliNB()).train(training_set),
    "SGDClassifier": SklearnClassifier(SGDClassifier()).train(training_set),
    "LinearSVC": SklearnClassifier(LinearSVC(dual=True)).train(training_set),
    "LogisticRegression": SklearnClassifier(LogisticRegression()).train(training_set),
}


# Evaluate classifiers
for name, clf in classifiers.items():
    print(f"{name} Accuracy: %{nltk.classify.accuracy(clf, testing_set) * 100}")

Naive Bayes Accuracy: %41.228070175438596
MultinomialNB Accuracy: %34.21052631578947
BernoulliNB Accuracy: %0.0
SGDClassifier Accuracy: %46.49122807017544
LinearSVC Accuracy: %39.473684210526315
LogisticRegression Accuracy: %35.96491228070175


In [34]:
# Voting Classifier
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = [clf.classify(features) for clf in self._classifiers]
        return mode(votes)

    def confidence(self, features):
        votes = []
        for classification in self._classifiers:
            vote = classification.classify(features)
            votes.append(vote)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

voted_classifier = VoteClassifier(*classifiers.values())
print(f"Voted Classifier Accuracy: {nltk.classify.accuracy(voted_classifier, testing_set) * 100}%")

ex_data = "zavallı"

print(f"Classification: {voted_classifier.classify(find_features(ex_data))}")
print(f"Confidence %: {voted_classifier.confidence(find_features(ex_data))*100}")

Voted Classifier Accuracy: 40.35087719298245%
Classification: V
Confidence %: 66.66666666666666


In [35]:
from zeyrek import MorphAnalyzer
import nltk
import logging

def run_examples():
    logging.getLogger("zeyrek.rulebasedanalyzer").setLevel(logging.ERROR)
    analyzer = MorphAnalyzer()
    
    with open('text.txt', encoding='utf-8') as text_file:
        text = text_file.read()

    tokens = tokenizer.tokenize(text)
    words = [
        token.content
        for token in tokens
        if token.content not in stop_words
    ]
    
    for word in words:
        possible_lemmas = analyzer.lemmatize(word)[0][1]
        if len(possible_lemmas) > 0:
            print(possible_lemmas[0]) 

if __name__ == '__main__':
    run_examples()

V
şekil
uç
üçgen
formasyon
geç
hadi
boleybol
oynamak
çocuk
üçgen
formasyon
oluşmak
gerekmek
.
voleybol
maç
yapmak
bir
nokta
bir
nokta
dümdüz
bir
çizgi
Çek
,
...
üçgen
formasyon
oluştururmusun
Dronların
çizgi
Üzer
gitmek
istemek
